In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers accelerate


Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())

2.6.0+cu124
CUDA available: True


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "rafalposwiata/deproberta-large-depression"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Model loaded successfully on", device) # just to make sure we have it not on CPU


Model loaded successfully on cuda


### So we can test the model now: 
I think it should work with this pretrained thing

In [5]:
text = "I can’t get out of bed and everything feels heavy."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

# Predict the thing
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    predicted_idx = torch.argmax(probs, dim=1).item()

label = model.config.id2label[predicted_idx]
confidence = probs[0][predicted_idx].item()

print(f"Input: {text}")
print(f"Prediction: {label}")
print(f"Confidence: {confidence:.3f}")

Input: I can’t get out of bed and everything feels heavy.
Prediction: moderate
Confidence: 0.513


So that works and should save me a lot of time. I don't think adding a regression head and retraining would be the play here, but I have it in my notes just in case. That will all go in another folder and I will start from here because I had some issues.
